In [1]:
from pathlib import Path
from models.transfersdataset import TransfersDataset
from models.lightgbm import LightGBM as Model

In [2]:
import pandas as pd
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
td.encode_last_positions()
td.encode()
td.drop()

In [5]:
def_features = ['duels90','duelswonpct','succssdefact90','defduels90','defduelswpct','aerduels90','aerduelspct','sltackles90','padjsltackles',
                'shotblocks90','intrcpt90','padjintrcpt','fouls90','ycards90','rcards90','passes90','accpassespct','fwdpasses90','accfwdpassespct',
                'bpasses90','accbpassespct','latpasses90','acclatpassespct','shortmedpasses90','accshortmedpassespct','longpasses90','acclongpassesct',
                'avgpasslen','avglongpasslen']

In [6]:
model_scores = pd.DataFrame()
def pipeline(pos, features=None, full_feature_set=False):
    td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
    td.filter_postion(pos)
    td.encode()
    td.drop()
    m = Model(data = td.data, features = features, full_feature_set=full_feature_set)
    display(m.data[m.features], features)
    m.train_test_split()
    m.scale()
    m.tune_hp()
    m.train()
    m.inverse_scale()
    global model_scores
    model_scores = pd.concat([model_scores, m.score(thresh=1.5)])
    # display(m.feature_importance())
    m.plot_predictions()
    display(m.top_n_predictions(20))
    display(m.top_n_predictions(20, worst=True))
    m.calculate_effectiveness()

In [7]:
pipeline('DEF')

,age,season,window,fee,club_from_elo,club_to_elo,league_from_elo,league_to_elo,marketval,matchesplayed,...,deepcomp90,deepcompcrosses90,prpasses90,accprpassespct,fk90,directfk90,directfkontrgtpct,corners90,pens,pencnvrspct
4,19,0.0,1,85.5,1816,1871,1429.238095,1561.400000,75.0,33,...,0.29,0.12,10.59,80.16,0.00,0.00,0.0,0.00,0,0.0
5,23,0.0,1,80.0,1877,1947,1671.233333,1605.100000,70.0,14,...,0.15,0.46,6.97,79.12,0.00,0.00,0.0,0.00,0,0.0
8,25,0.0,1,65.0,1871,2041,1561.400000,1696.800000,55.0,25,...,1.33,2.02,10.65,78.23,0.00,0.00,0.0,0.52,0,0.0
12,21,0.0,1,55.0,1747,1842,1696.800000,1696.800000,35.0,35,...,0.35,0.48,8.61,66.67,0.00,0.00,0.0,0.00,0,0.0
14,21,0.0,1,50.0,1816,1840,1475.571429,1671.233333,50.0,29,...,0.35,0.29,8.97,77.86,0.03,0.03,0.0,0.00,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,29,4.0,1,0.0,1597,1489,1630.900000,1435.363636,1.3,29,...,0.03,0.29,8.54,62.88,0.00,0.00,0.0,0.00,0,0.0
2214,18,4.0,0,17.0,1612,1696,1474.736842,1552.633333,2.8,22,...,0.73,0.94,4.19,72.50,0.00,0.00,0.0,0.21,0,0.0
2216,24,4.0,0,8.5,1608,1759,1630.900000,1630.900000,6.0,33,...,0.09,0.03,6.02,62.63,0.03,0.00,0.0,0.00,0,0.0
2218,25,4.0,0,2.7,1608,1621,1630.900000,1653.333333,3.0,16,...,0.19,0.06,5.95,69.15,0.00,0.00,0.0,0.00,0,0.0


None

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'num_leaves': 69, 'learning_rate': 0.05971064016381608, 'subsample': 0.9975829612462166, 'colsample_bytree': 0.5161786838426399, 'min_child_samples': 27, 'reg_alpha': 9.58918753428264, 'reg_lambda': 6.726610897579423}
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015897 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10205
[LightGBM] [Info] Number of data points in the train set: 563, number of used features: 89
thresh = 1.5
mae: 2.6363482357687418
mape: 0.7871354304952225
mae_below_thresh: 1.5240572690527125
mape_above_thresh: 0.45287967597312456


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
595,Giuseppe Pezzella,22,20/21,Italy,IT1,Udinese Calcio,Italy,IT1,Parma Calcio 1913,1,2.4,6.0,5.75,6.0,5.998294,0.001706
1666,Kévin Rodrigues,28,22/23,Spain,ES1,Real Sociedad,Türkiye,TR1,Adana Demirspor,1,1.8,2.3,0.00,2.3,2.274376,0.025624
621,Steven Zuber,28,20/21,Germany,L1,TSG 1899 Hoffenheim,Germany,L1,Eintracht Frankfurt,1,4.5,3.5,3.00,3.5,3.548249,0.048249
1048,Dimitri Foulquier,28,21/22,Spain,ES1,Granada CF,Spain,ES1,Valencia CF,1,3.5,4.0,2.50,4.0,4.053494,0.053494
38,Kieran Trippier,28,19/20,England,GB1,Tottenham Hotspur,Spain,ES1,Atlético de Madrid,1,35.0,28.0,22.00,28.0,27.933571,0.066429
796,Jeremie Frimpong,20,20/21,Scotland,SC1,Celtic FC,Germany,L1,Bayer 04 Leverkusen,0,3.0,10.0,11.00,10.0,10.175212,0.175212
1108,Bart Nieuwkoop,25,21/22,Netherlands,NL1,Feyenoord Rotterdam,Belgium,BE1,Union Saint-Gilloise,1,2.2,2.5,0.00,2.5,2.312982,0.187018
2059,Matt Doherty,31,23/24,Spain,ES1,Atlético de Madrid,England,GB1,Wolverhampton Wanderers,1,5.0,4.0,0.00,4.0,3.809628,0.190372
2096,Portu,31,23/24,Spain,ES1,Real Sociedad,Spain,ES1,Getafe CF,1,3.5,3.5,3.50,3.5,3.704925,0.204925
1550,Michal Sadílek,23,22/23,Netherlands,NL1,PSV Eindhoven,Netherlands,NL1,Twente Enschede FC,1,4.0,4.0,1.50,4.0,4.210039,0.210039


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
875,Fikayo Tomori,23,21/22,England,GB1,Chelsea FC,Italy,IT1,AC Milan,1,28.0,50.0,34.40,50.0,34.110755,15.889245
1205,Reinildo Mandava,28,21/22,France,FR1,LOSC Lille,Spain,ES1,Atlético de Madrid,0,9.0,22.0,3.00,22.0,6.502063,15.497937
1317,Robin Gosens,27,22/23,Italy,IT1,Atalanta BC,Italy,IT1,Inter Milan,1,28.0,15.0,27.40,15.0,28.128188,13.128188
1341,Pervis Estupiñán,24,22/23,Spain,ES1,Villarreal CF,England,GB1,Brighton & Hove Albion,1,20.0,32.0,17.80,32.0,20.052769,11.947231
1310,Niklas Süle,26,22/23,Germany,L1,Bayern Munich,Germany,L1,Borussia Dortmund,1,35.0,35.0,0.00,35.0,24.093871,10.906129
1833,Pedro Porro,23,23/24,Portugal,PO1,Sporting CP,England,GB1,Tottenham Hotspur,1,40.0,45.0,40.00,45.0,34.856978,10.143022
397,Julian Weigl,24,19/20,Germany,L1,Borussia Dortmund,Portugal,PO1,SL Benfica,0,25.0,18.5,20.00,18.5,28.458024,9.958024
485,Thomas Meunier,28,20/21,France,FR1,Paris Saint-Germain,Germany,L1,Borussia Dortmund,1,24.0,12.0,0.00,12.0,21.711666,9.711666
290,Sofyan Amrabat,23,19/20,Belgium,BE1,Club Brugge KV,Italy,IT1,Hellas Verona,0,3.0,14.5,3.65,14.5,5.036225,9.463775
562,Alexis Saelemaekers,21,20/21,Belgium,BE1,RSC Anderlecht,Italy,IT1,AC Milan,1,6.3,18.0,7.65,18.0,8.897051,9.102949


In [8]:
pipeline('DEF', def_features)

,age,season,window,fee,club_from_elo,club_to_elo,league_from_elo,league_to_elo,marketval,matchesplayed,...,bpasses90,accbpassespct,latpasses90,acclatpassespct,shortmedpasses90,accshortmedpassespct,longpasses90,acclongpassesct,avgpasslen,avglongpasslen
4,19,0.0,1,85.5,1816,1871,1429.238095,1561.400000,75.0,33,...,4.49,97.44,28.80,97.00,55.93,95.16,4.75,57.58,21.02,35.89
5,23,0.0,1,80.0,1877,1947,1671.233333,1605.100000,70.0,14,...,5.06,89.39,11.49,94.00,33.70,92.95,4.21,49.09,19.57,32.68
8,25,0.0,1,65.0,1871,2041,1561.400000,1696.800000,55.0,25,...,8.76,94.12,21.21,82.79,47.66,91.26,4.47,62.50,19.02,33.24
12,21,0.0,1,55.0,1747,1842,1696.800000,1696.800000,35.0,35,...,3.67,86.86,5.69,77.36,26.98,83.10,4.02,44.00,17.70,34.77
14,21,0.0,1,50.0,1816,1840,1475.571429,1671.233333,50.0,29,...,2.95,95.65,21.15,93.48,38.16,92.61,6.31,52.28,22.59,40.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,29,4.0,1,0.0,1597,1489,1630.900000,1435.363636,1.3,29,...,4.30,97.74,8.41,83.08,25.65,85.25,5.73,41.24,21.07,34.07
2214,18,4.0,0,17.0,1612,1696,1474.736842,1552.633333,2.8,22,...,12.03,93.91,10.67,89.22,33.38,89.97,0.63,66.67,14.22,7.15
2216,24,4.0,0,8.5,1608,1759,1630.900000,1630.900000,6.0,33,...,1.79,93.22,12.53,95.87,24.67,86.31,4.08,57.46,21.25,35.49
2218,25,4.0,0,2.7,1608,1621,1630.900000,1653.333333,3.0,16,...,3.80,100.00,18.87,93.96,32.49,92.40,5.32,52.38,22.80,38.82


['duels90',
 'duelswonpct',
 'succssdefact90',
 'defduels90',
 'defduelswpct',
 'aerduels90',
 'aerduelspct',
 'sltackles90',
 'padjsltackles',
 'shotblocks90',
 'intrcpt90',
 'padjintrcpt',
 'fouls90',
 'ycards90',
 'rcards90',
 'passes90',
 'accpassespct',
 'fwdpasses90',
 'accfwdpassespct',
 'bpasses90',
 'accbpassespct',
 'latpasses90',
 'acclatpassespct',
 'shortmedpasses90',
 'accshortmedpassespct',
 'longpasses90',
 'acclongpassesct',
 'avgpasslen',
 'avglongpasslen']

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'num_leaves': 58, 'learning_rate': 0.09936863535707999, 'subsample': 0.5728822860156043, 'colsample_bytree': 0.6876192066567846, 'min_child_samples': 19, 'reg_alpha': 1.829562481561853, 'reg_lambda': 1.1076783261820964}
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002696 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5597
[LightGBM] [Info] Number of data points in the train set: 563, number of used features: 42
thresh = 1.5
mae: 2.4652028698846684
mape: 0.5898602736580106
mae_below_thresh: 0.9438104751496974
mape_above_thresh: 0.40410674915059125


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
288,Sergey Parshivlyuk,30,19/20,Russia,RU1,FK Rostov,Russia,RU1,Dinamo Moscow,1,3.00,1.8,0.00,1.8,1.791151,0.008849
344,Fedor Kudryashov,32,19/20,Türkiye,TR1,Basaksehir FK,Russia,RU1,FC Sochi,1,1.75,1.2,0.00,1.2,1.165057,0.034943
278,Reinildo Mandava,25,19/20,Portugal,PO1,B SAD,France,FR1,LOSC Lille,1,0.60,2.4,3.00,2.4,2.359811,0.040189
1020,Jack Hendry,26,21/22,Belgium,BE1,KV Oostende,Belgium,BE1,Club Brugge KV,1,3.50,5.0,4.50,5.0,4.931217,0.068783
865,Angeliño,24,21/22,England,GB1,Manchester City,Germany,L1,RB Leipzig,1,35.00,30.0,18.00,30.0,30.105520,0.105520
1666,Kévin Rodrigues,28,22/23,Spain,ES1,Real Sociedad,Türkiye,TR1,Adana Demirspor,1,1.80,2.3,0.00,2.3,2.187530,0.112470
362,Pierre-Yves Polomat,25,19/20,France,FR1,AS Saint-Étienne,Türkiye,TR1,Genclerbirligi Ankara,1,1.50,0.9,0.00,0.9,1.015381,0.115381
621,Steven Zuber,28,20/21,Germany,L1,TSG 1899 Hoffenheim,Germany,L1,Eintracht Frankfurt,1,4.50,3.5,3.00,3.5,3.354771,0.145229
1108,Bart Nieuwkoop,25,21/22,Netherlands,NL1,Feyenoord Rotterdam,Belgium,BE1,Union Saint-Gilloise,1,2.20,2.5,0.00,2.5,2.652188,0.152188
595,Giuseppe Pezzella,22,20/21,Italy,IT1,Udinese Calcio,Italy,IT1,Parma Calcio 1913,1,2.40,6.0,5.75,6.0,6.157733,0.157733


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
1317,Robin Gosens,27,22/23,Italy,IT1,Atalanta BC,Italy,IT1,Inter Milan,1,28.0,15.0,27.40,15.0,29.618003,14.618003
1205,Reinildo Mandava,28,21/22,France,FR1,LOSC Lille,Spain,ES1,Atlético de Madrid,0,9.0,22.0,3.00,22.0,7.651449,14.348551
485,Thomas Meunier,28,20/21,France,FR1,Paris Saint-Germain,Germany,L1,Borussia Dortmund,1,24.0,12.0,0.00,12.0,26.044000,14.044000
875,Fikayo Tomori,23,21/22,England,GB1,Chelsea FC,Italy,IT1,AC Milan,1,28.0,50.0,34.40,50.0,37.546533,12.453467
1341,Pervis Estupiñán,24,22/23,Spain,ES1,Villarreal CF,England,GB1,Brighton & Hove Albion,1,20.0,32.0,17.80,32.0,20.181836,11.818164
290,Sofyan Amrabat,23,19/20,Belgium,BE1,Club Brugge KV,Italy,IT1,Hellas Verona,0,3.0,14.5,3.65,14.5,4.010040,10.489960
562,Alexis Saelemaekers,21,20/21,Belgium,BE1,RSC Anderlecht,Italy,IT1,AC Milan,1,6.3,18.0,7.65,18.0,9.394822,8.605178
1744,Julian Ryerson,25,22/23,Germany,L1,1.FC Union Berlin,Germany,L1,Borussia Dortmund,0,4.0,13.0,5.00,13.0,5.222821,7.777179
139,Denis Suárez,25,19/20,Spain,ES1,FC Barcelona,Spain,ES1,Celta de Vigo,1,12.0,9.5,12.90,9.5,17.220120,7.720120
8,João Cancelo,25,19/20,Italy,IT1,Juventus FC,England,GB1,Manchester City,1,55.0,36.0,65.00,36.0,43.434144,7.434144


In [9]:
scores = pd.read_csv('lightgbm_def.csv').iloc[:,1:].sort_values('MSE')
scores.Features = scores.Features.str.split(', ')
feature_sets = scores.Features.head(10).to_dict()

In [10]:
for _, features in feature_sets.items():
    pipeline(pos='DEF', features=features, full_feature_set=True)

,age,season,window,fee,club_from_elo,club_to_elo,league_from_elo,league_to_elo,marketval,matchesplayed,...,pencnvrspct,acccrossesleftpct,kp90,fk90,corners90,shotblocks90,accbpassespct,rcvdpasses90,aerduelspct,sltackles90
4,19,0.0,1,85.5,1816,1871,1429.238095,1561.400000,75.0,33,...,0.0,50.00,0.12,0.00,0.00,0.75,97.44,43.82,54.84,0.14
5,23,0.0,1,80.0,1877,1947,1671.233333,1605.100000,70.0,14,...,0.0,37.50,0.08,0.00,0.00,0.54,89.39,27.34,57.89,0.92
8,25,0.0,1,65.0,1871,2041,1561.400000,1696.800000,55.0,25,...,0.0,50.00,0.52,0.00,0.52,0.21,94.12,42.04,36.84,0.34
12,21,0.0,1,55.0,1747,1842,1696.800000,1696.800000,35.0,35,...,0.0,0.00,0.13,0.00,0.00,0.38,86.86,19.26,48.31,2.63
14,21,0.0,1,50.0,1816,1840,1475.571429,1671.233333,50.0,29,...,0.0,0.00,0.13,0.03,0.00,0.48,95.65,30.34,73.04,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,29,4.0,1,0.0,1597,1489,1630.900000,1435.363636,1.3,29,...,0.0,25.00,0.06,0.00,0.00,0.68,97.74,22.74,59.78,0.13
2214,18,4.0,0,17.0,1612,1696,1474.736842,1552.633333,2.8,22,...,0.0,33.33,0.84,0.00,0.21,0.00,93.91,27.10,11.11,0.52
2216,24,4.0,0,8.5,1608,1759,1630.900000,1630.900000,6.0,33,...,0.0,0.00,0.00,0.03,0.00,0.40,93.22,14.51,60.67,0.15
2218,25,4.0,0,2.7,1608,1621,1630.900000,1653.333333,3.0,16,...,0.0,0.00,0.00,0.00,0.00,0.76,100.00,23.88,37.50,0.06


['age',
 'season',
 'window',
 'fee',
 'club_from_elo',
 'club_to_elo',
 'league_from_elo',
 'league_to_elo',
 'marketval',
 'matchesplayed',
 'minsplayed',
 'foot',
 'height',
 'weight',
 'succssdribblespct',
 'bpasses90',
 'padjsltackles',
 'hdrgoals90',
 'acclatpassespct',
 'accpassespct',
 'succssdefact90',
 'accpenareapassespct',
 'crossesleft90',
 'rcards90',
 'directfk90',
 'directfkontrgtpct',
 'pens',
 'pencnvrspct',
 'acccrossesleftpct',
 'kp90',
 'fk90',
 'corners90',
 'shotblocks90',
 'accbpassespct',
 'rcvdpasses90',
 'aerduelspct',
 'sltackles90']

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'num_leaves': 40, 'learning_rate': 0.03817701463638031, 'subsample': 0.963749327607139, 'colsample_bytree': 0.7547353802172032, 'min_child_samples': 26, 'reg_alpha': 2.4797683376761475, 'reg_lambda': 9.957676379396133}
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014055 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3091
[LightGBM] [Info] Number of data points in the train set: 563, number of used features: 34
thresh = 1.5
mae: 2.4895082467140472
mape: 0.7307428336674354
mae_below_thresh: 1.4519697954600528
mape_above_thresh: 0.3994692196479922


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
1032,Tin Jedvaj,25,21/22,Germany,L1,Bayer 04 Leverkusen,Russia,RU1,Lokomotiv Moscow,1,3.0,4.0,4.00,4.0,4.013614,0.013614
222,Daley Sinkgraven,23,19/20,Netherlands,NL1,Ajax Amsterdam,Germany,L1,Bayer 04 Leverkusen,1,3.0,4.8,5.00,4.8,4.757091,0.042909
1666,Kévin Rodrigues,28,22/23,Spain,ES1,Real Sociedad,Türkiye,TR1,Adana Demirspor,1,1.8,2.3,0.00,2.3,2.353716,0.053716
1108,Bart Nieuwkoop,25,21/22,Netherlands,NL1,Feyenoord Rotterdam,Belgium,BE1,Union Saint-Gilloise,1,2.2,2.5,0.00,2.5,2.441085,0.058915
796,Jeremie Frimpong,20,20/21,Scotland,SC1,Celtic FC,Germany,L1,Bayer 04 Leverkusen,0,3.0,10.0,11.00,10.0,9.885480,0.114520
264,Victor Nelsson,20,19/20,Denmark,DK1,FC Nordsjaelland,Denmark,DK1,FC Copenhagen,1,3.0,4.5,3.60,4.5,4.632816,0.132816
173,Alberto Moreno,27,19/20,England,GB1,Liverpool FC,Spain,ES1,Villarreal CF,1,9.0,6.0,0.00,6.0,6.188632,0.188632
1633,Jure Balkovec,27,22/23,Türkiye,TR1,Fatih Karagümrük,Türkiye,TR1,Alanyaspor,1,2.3,2.2,0.00,2.2,2.455506,0.255506
1048,Dimitri Foulquier,28,21/22,Spain,ES1,Granada CF,Spain,ES1,Valencia CF,1,3.5,4.0,2.50,4.0,3.712019,0.287981
1756,Samet Akaydin,28,22/23,Türkiye,TR1,Adana Demirspor,Türkiye,TR1,Fenerbahce,0,2.0,4.2,3.70,4.2,3.892793,0.307207


...

['age',
 'season',
 'window',
 'fee',
 'club_from_elo',
 'club_to_elo',
 'league_from_elo',
 'league_to_elo',
 'marketval',
 'matchesplayed',
 'minsplayed',
 'foot',
 'height',
 'weight',
 'succssdribblespct',
 'bpasses90',
 'padjsltackles',
 'hdrgoals90',
 'acclatpassespct',
 'accpassespct',
 'succssdefact90',
 'accpenareapassespct',
 'crossesleft90',
 'rcards90',
 'directfk90',
 'directfkontrgtpct']

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'num_leaves': 5, 'learning_rate': 0.07170177273241819, 'subsample': 0.730275603650009, 'colsample_bytree': 0.7403120570665898, 'min_child_samples': 28, 'reg_alpha': 5.336709729005886, 'reg_lambda': 2.203074192862282}
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002578 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2143
[LightGBM] [Info] Number of data points in the train set: 563, number of used features: 25
thresh = 1.5
mae: 2.4893330474244393
mape: 0.6468608962247292
mae_below_thresh: 1.205128986596761
mape_above_thresh: 0.38074792683826336


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
2096,Portu,31,23/24,Spain,ES1,Real Sociedad,Spain,ES1,Getafe CF,1,3.50,3.5,3.50,3.5,3.502708,0.002708
639,Filip Novak,30,20/21,Türkiye,TR1,Trabzonspor,Türkiye,TR1,Fenerbahce,1,2.80,2.0,0.00,2.0,2.015635,0.015635
595,Giuseppe Pezzella,22,20/21,Italy,IT1,Udinese Calcio,Italy,IT1,Parma Calcio 1913,1,2.40,6.0,5.75,6.0,6.029794,0.029794
913,Maxwel Cornet,24,21/22,France,FR1,Olympique Lyon,England,GB1,Burnley FC,1,12.00,15.0,15.00,15.0,15.048049,0.048049
1633,Jure Balkovec,27,22/23,Türkiye,TR1,Fatih Karagümrük,Türkiye,TR1,Alanyaspor,1,2.30,2.2,0.00,2.2,2.137641,0.062359
222,Daley Sinkgraven,23,19/20,Netherlands,NL1,Ajax Amsterdam,Germany,L1,Bayer 04 Leverkusen,1,3.00,4.8,5.00,4.8,4.733585,0.066415
260,Neven Subotic,30,19/20,France,FR1,AS Saint-Étienne,Germany,L1,1.FC Union Berlin,1,4.00,2.0,0.00,2.0,2.079015,0.079015
1075,Antonio Candreva,34,21/22,Italy,IT1,Inter Milan,Italy,IT1,UC Sampdoria,1,2.50,2.5,2.50,2.5,2.589759,0.089759
264,Victor Nelsson,20,19/20,Denmark,DK1,FC Nordsjaelland,Denmark,DK1,FC Copenhagen,1,3.00,4.5,3.60,4.5,4.604816,0.104816
1108,Bart Nieuwkoop,25,21/22,Netherlands,NL1,Feyenoord Rotterdam,Belgium,BE1,Union Saint-Gilloise,1,2.20,2.5,0.00,2.5,2.361729,0.138271


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
1317,Robin Gosens,27,22/23,Italy,IT1,Atalanta BC,Italy,IT1,Inter Milan,1,28.0,15.0,27.40,15.0,31.334597,16.334597
485,Thomas Meunier,28,20/21,France,FR1,Paris Saint-Germain,Germany,L1,Borussia Dortmund,1,24.0,12.0,0.00,12.0,26.945492,14.945492
1205,Reinildo Mandava,28,21/22,France,FR1,LOSC Lille,Spain,ES1,Atlético de Madrid,0,9.0,22.0,3.00,22.0,9.276993,12.723007
875,Fikayo Tomori,23,21/22,England,GB1,Chelsea FC,Italy,IT1,AC Milan,1,28.0,50.0,34.40,50.0,37.441336,12.558664
1341,Pervis Estupiñán,24,22/23,Spain,ES1,Villarreal CF,England,GB1,Brighton & Hove Albion,1,20.0,32.0,17.80,32.0,20.428525,11.571475
397,Julian Weigl,24,19/20,Germany,L1,Borussia Dortmund,Portugal,PO1,SL Benfica,0,25.0,18.5,20.00,18.5,29.826232,11.326232
290,Sofyan Amrabat,23,19/20,Belgium,BE1,Club Brugge KV,Italy,IT1,Hellas Verona,0,3.0,14.5,3.65,14.5,3.897079,10.602921
795,Shkodran Mustafi,28,20/21,England,GB1,Arsenal FC,Germany,L1,FC Schalke 04,0,15.0,2.5,0.00,2.5,11.169662,8.669662
562,Alexis Saelemaekers,21,20/21,Belgium,BE1,RSC Anderlecht,Italy,IT1,AC Milan,1,6.3,18.0,7.65,18.0,9.411043,8.588957
879,Emerson Royal,22,21/22,Spain,ES1,Real Betis Balompié,Spain,ES1,FC Barcelona,1,25.0,25.0,14.00,25.0,33.411194,8.411194


In [11]:
with pd.option_context('display.max_colwidth', None):
    display(model_scores.sort_values('mape_above_thresh'))

,mae,mape,mae_below_thresh,mape_above_thresh,features
0,2.489333,0.646861,1.205129,0.380748,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, succssdribblespct, bpasses90, padjsltackles, hdrgoals90, acclatpassespct, accpassespct, succssdefact90, accpenareapassespct, crossesleft90, rcards90, directfk90, directfkontrgtpct]"
0,2.505552,0.672291,1.277214,0.390239,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, succssdribblespct, bpasses90, padjsltackles, hdrgoals90, acclatpassespct, accpassespct, succssdefact90, accpenareapassespct, crossesleft90, rcards90, directfk90, directfkontrgtpct, pens, pencnvrspct, acccrossesleftpct, kp90, fk90, corners90, shotblocks90, accbpassespct, rcvdpasses90]"
0,2.489508,0.730743,1.451970,0.399469,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, succssdribblespct, bpasses90, padjsltackles, hdrgoals90, acclatpassespct, accpassespct, succssdefact90, accpenareapassespct, crossesleft90, rcards90, directfk90, directfkontrgtpct, pens, pencnvrspct, acccrossesleftpct, kp90, fk90, corners90, shotblocks90, accbpassespct, rcvdpasses90, aerduelspct, sltackles90]"
0,2.465203,0.589860,0.943810,0.404107,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, duels90, duelswonpct, succssdefact90, defduels90, defduelswpct, aerduels90, aerduelspct, sltackles90, padjsltackles, shotblocks90, intrcpt90, padjintrcpt, fouls90, ycards90, rcards90, passes90, accpassespct, fwdpasses90, accfwdpassespct, bpasses90, accbpassespct, latpasses90, acclatpassespct, shortmedpasses90, accshortmedpassespct, longpasses90, acclongpassesct, avgpasslen, avglongpasslen]"
0,2.560184,0.754783,1.508851,0.406998,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, succssdribblespct, bpasses90, padjsltackles, hdrgoals90, acclatpassespct, accpassespct, succssdefact90, accpenareapassespct, crossesleft90, rcards90, directfk90, directfkontrgtpct, pens, pencnvrspct, acccrossesleftpct, kp90, fk90, corners90, shotblocks90, accbpassespct, rcvdpasses90, aerduelspct, sltackles90, thirdassist90, intrcpt90, deepcompcrosses90]"
0,2.455812,0.715091,1.360970,0.409255,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, succssdribblespct, bpasses90, padjsltackles, hdrgoals90, acclatpassespct, accpassespct, succssdefact90, accpenareapassespct, crossesleft90, rcards90, directfk90, directfkontrgtpct, pens, pencnvrspct, acccrossesleftpct, kp90, fk90, corners90, shotblocks90, accbpassespct, rcvdpasses90, aerduelspct, sltackles90, thirdassist90]"
0,2.443367,0.648413,1.118589,0.417932,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, succssdribblespct, bpasses90, padjsltackles, hdrgoals90, acclatpassespct, accpassespct, succssdefact90, accpenareapassespct, crossesleft90, rcards90, directfk90]"
0,2.529867,0.673885,1.187972,0.418015,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, succssdribblespct, bpasses90, padjsltackles, hdrgoals90, acclatpassespct, accpassespct, succssdefact90, accpenareapassespct, crossesleft90, rcards90, directfk90, directfkontrgtpct, pens]"
0,2.498141,0.624287,1.059390,0.419590,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, succssdribblespct, bpasses90, padjsltackles, hdrgoals90, acclat